This code shows how a theoretically optimal QME would perform. Instead of using LPME to estimate the normalized gradient, we compute the true (normalized) gradient and use it directly.

The resulting squared error should *always* be $ \approx $ 0, even on inputs that are not well-formed.

In [1]:
import numpy as np

import sys
sys.path.append('../')
from common import Sphere, create_a_B, normalize
from qme import QMESC

In [2]:
np.random.seed(7)

nc = 3 # number of classes
rB = 0.5 # radius of big circle
rS = 0.2 # radius of small circle

In [3]:
q = nc ** 2 - nc # number of off-diag elements
sphere = Sphere(np.random.randn(q), rB, q)

In [4]:
a, B = create_a_B(sphere, q, well_formed=False)
d = np.array(a + B @ sphere.origin)[0] # see paper for an explanation

In [5]:
a

array([0.16151099, 0.21945779, 0.2596864 , 0.12308128, 0.0213039 ,
       0.09309924])

In [6]:
B

matrix([[ 0.0918981 ,  0.01367911, -0.01464161, -0.03374885, -0.01594855,
          0.00542377],
        [ 0.01367911,  0.2460313 , -0.16528713, -0.36697316, -0.06626116,
          0.0702057 ],
        [-0.01464161, -0.16528713,  0.19526496,  0.30098713,  0.11494059,
         -0.04367302],
        [-0.03374885, -0.36697316,  0.30098713,  0.80389665,  0.1710397 ,
         -0.09867968],
        [-0.01594855, -0.06626116,  0.11494059,  0.1710397 ,  0.10597845,
         -0.03141615],
        [ 0.00542377,  0.0702057 , -0.04367302, -0.09867968, -0.03141615,
          0.0507994 ]])

In [7]:
# f_z and f_neg0 are special, see paper for explanation
f_z = normalize(d)

z_neg0 = np.zeros(q)
z_neg0[0] = -(rB - rS)
f_neg0 = normalize(np.array(d + B @ z_neg0)[0])

# f_0 ... f_q-1, compute true derivative and store normalized "slope" (LPME recovers this)
fs = []
for i in range(0, q):
    z_i = np.zeros(q)
    z_i[i] = (rB - rS)
    f_i = normalize(np.array(d + B @ z_i)[0])
    fs.append(f_i)

In [8]:
qm = QMESC(sphere, rS, f_z, f_neg0, fs, well_formed=False)
ahat, Bhat = qm.compute_a_b()

In [9]:
a

array([0.16151099, 0.21945779, 0.2596864 , 0.12308128, 0.0213039 ,
       0.09309924])

In [10]:
ahat

array([0.16151099, 0.21945779, 0.2596864 , 0.12308128, 0.0213039 ,
       0.09309924])

In [11]:
B

matrix([[ 0.0918981 ,  0.01367911, -0.01464161, -0.03374885, -0.01594855,
          0.00542377],
        [ 0.01367911,  0.2460313 , -0.16528713, -0.36697316, -0.06626116,
          0.0702057 ],
        [-0.01464161, -0.16528713,  0.19526496,  0.30098713,  0.11494059,
         -0.04367302],
        [-0.03374885, -0.36697316,  0.30098713,  0.80389665,  0.1710397 ,
         -0.09867968],
        [-0.01594855, -0.06626116,  0.11494059,  0.1710397 ,  0.10597845,
         -0.03141615],
        [ 0.00542377,  0.0702057 , -0.04367302, -0.09867968, -0.03141615,
          0.0507994 ]])

In [12]:
Bhat

matrix([[ 0.0918981 ,  0.01367911, -0.01464161, -0.03374885, -0.01594855,
          0.00542377],
        [ 0.01367911,  0.2460313 , -0.16528713, -0.36697316, -0.06626116,
          0.0702057 ],
        [-0.01464161, -0.16528713,  0.19526496,  0.30098713,  0.11494059,
         -0.04367302],
        [-0.03374885, -0.36697316,  0.30098713,  0.80389665,  0.1710397 ,
         -0.09867968],
        [-0.01594855, -0.06626116,  0.11494059,  0.1710397 ,  0.10597845,
         -0.03141615],
        [ 0.00542377,  0.0702057 , -0.04367302, -0.09867968, -0.03141615,
          0.0507994 ]])

In [13]:
# ahat error
np.linalg.norm(ahat - a)

1.1292503550707428e-15

In [14]:
# bhat error
np.linalg.norm(Bhat - B, ord='fro')

2.3326595404255333e-15